In [1]:
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime

# a little hacky, but works if you don't want to actually install the
# custom packages 
sys.path.append('../')
from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.utils import pool_init_price, solve_for_liquidity_delta
from uniswapv3_simulator.math import tick_to_sqrt_price, sqrt_price_to_tick
from utils import amount_to_float

In [2]:
POOL = 'USDC-WETH-500'
MAX_DATE = '2022-01-28'

timestamp = datetime.now().strftime('%y%m%d%H%M%S')
logging.basicConfig(level=logging.INFO,
                    filename=f'./logs/{POOL}_{timestamp}.log')
logging.getLogger('uniswap-v3').setLevel(logging.DEBUG)
logging.getLogger('covalent_api').setLevel(logging.DEBUG)

In [3]:
data = pd.read_pickle('../data/pool_data_clean.pickle')
swaps = data[POOL]['swaps']
liquidity = data[POOL]['liquidity']

In [4]:
swaps.info()
swaps.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247617 entries, 0 to 1247616
Data columns (total 18 columns):
 #   Column                          Non-Null Count    Dtype              
---  ------                          --------------    -----              
 0   swap_time                       1247617 non-null  datetime64[ns, UTC]
 1   tx_hash                         1247617 non-null  string             
 2   token_0_amount                  1247617 non-null  float64            
 3   token_1_amount                  1247617 non-null  float64            
 4   pool_address                    1247617 non-null  string             
 5   price_tick                      1247617 non-null  int64              
 6   price_tick_adjusted             1247617 non-null  float64            
 7   contract_address_token_0        1247617 non-null  string             
 8   contract_name_token_0           1247617 non-null  string             
 9   contract_ticker_symbol_token_0  1247617 non-null  string 

,swap_time,tx_hash,token_0_amount,token_1_amount,pool_address,price_tick,price_tick_adjusted,contract_address_token_0,contract_name_token_0,contract_ticker_symbol_token_0,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,pool_fee,pool_tick_spacing,pool_deploy_time
0,2022-01-30 04:52:19+00:00,0x36621c4daf5223d66e6b95b71e980e2135201db0c154...,-129.038949,0.050000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,197756,0.000387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,500,10,2021-05-05 21:42:11+00:00
1,2022-01-30 04:52:19+00:00,0xf78b59b49af825cb4a8bddc5de096fdacc057e67ef89...,-25807.129150,10.000000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,197757,0.000387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,500,10,2021-05-05 21:42:11+00:00
2,2022-01-30 04:48:53+00:00,0x432fd168796a6a60378e5b2dd517669f3561d30c785e...,-19691.729042,7.630000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,197756,0.000387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,500,10,2021-05-05 21:42:11+00:00
3,2022-01-30 04:47:53+00:00,0xee6a0812c1fef124feab8a3d16eb8746a42296de4e43...,297650.337110,-115.247573,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,197756,0.000387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,500,10,2021-05-05 21:42:11+00:00
4,2022-01-30 04:47:30+00:00,0xc1d9409536ab979f2f3c398a08b233320c742b1409c2...,-27219.066547,10.552313,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,197762,0.000387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,500,10,2021-05-05 21:42:11+00:00


In [5]:
liquidity.info()
liquidity.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31210 entries, 0 to 31209
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   txn_time                        31210 non-null  datetime64[ns, UTC]
 1   tx_hash                         31210 non-null  string             
 2   token_0_amount                  31210 non-null  float64            
 3   token_1_amount                  31210 non-null  float64            
 4   pool_address                    31210 non-null  string             
 5   liquidity_event                 31210 non-null  string             
 6   price_tick_lower                31210 non-null  int64              
 7   price_tick_upper                31210 non-null  int64              
 8   price_tick_lower_adjusted       31210 non-null  float64            
 9   price_tick_upper_adjusted       31210 non-null  float64            
 10  contract_a

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2022-01-29 18:52:48+00:00,0xaac213e229e4587d4981d05bbab06da2f7c64c70139e...,1212.627978,2.090520,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,196910,197880,0.000356,0.000392,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.002733
1,2022-01-29 18:52:46+00:00,0x27993e06ca64d24bfbfa67993ac425b48b3d647085c4...,45631.897804,78.669352,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,196910,197880,0.000356,0.000392,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.102843
2,2022-01-29 18:52:09+00:00,0xd2e45378cb7051760d5684343abec778e761044b26ed...,256006.205333,0.000000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,REMOVE_LIQUIDITY,197730,198660,0.000386,0.000424,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Burn,500,10,2021-05-05 21:42:11+00:00,-0.110743
3,2022-01-29 18:48:47+00:00,0xd274375a0422f6c1dac70bd7d9a225b458fe0f1f6d97...,169559.787040,90.000000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,197380,197970,0.000373,0.000396,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.270260
4,2022-01-29 18:39:34+00:00,0xafb3ad5c97e4d0300c62e5877530666809b322d53134...,1216.740422,0.399859,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,193380,203190,0.000250,0.000667,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.000102


In [6]:
adds = liquidity.loc[liquidity['liquidity_event'] == 'ADD_LIQUIDITY', :].copy()
adds = adds.sort_values('txn_time').reset_index(drop=True)
adds.head()

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2021-05-05 21:42:11+00:00,0x125e0b641d4a4b08806bf52c0c6757648c9963bcda86...,2995.507735,1.000000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,191150,198080,0.000200,0.000400,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,500,10,2021-05-05 21:42:11+00:00,0.000345
1,2021-05-05 22:44:11+00:00,0xa564ea4677bda271bbb2eebec9774577b9b101c46eaf...,1.000000,0.000146,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,194990,195010,0.000294,0.000294,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,500,10,2021-05-05 21:42:11+00:00,0.000026
2,2021-05-05 23:34:39+00:00,0xef93747acabe06ab11f663321f0de466bf9ae35888ba...,598.469729,2.600000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,186730,195460,0.000129,0.000308,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,500,10,2021-05-05 21:42:11+00:00,0.000448
3,2021-05-05 23:44:49+00:00,0x5f066eb8fefba0d1b491ffb924b708dc24158192a77f...,3405.610348,0.000000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,194660,194990,0.000284,0.000294,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.003508
4,2021-05-05 23:48:19+00:00,0x5184d8d442ac1696e9f7fb89c31db8c1de8ae9c80f3e...,1549.658850,0.151653,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,ADD_LIQUIDITY,194640,194660,0.000284,0.000284,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,500,10,2021-05-05 21:42:11+00:00,0.035119


In [7]:
first_add_hash = adds.at[0, 'tx_hash']
print(f'First liquidity add hash: {first_add_hash}')

First liquidity add hash: 0x125e0b641d4a4b08806bf52c0c6757648c9963bcda8681e4f996f09e00d4c2cc


In [8]:
# from https://etherscan.io/tx/0x125e0b641d4a4b08806bf52c0c6757648c9963bcda8681e4f996f09e00d4c2cc#eventlog
liquidity_delta = amount_to_float('345073104699360', 18)  # belive all liquidity amounts use 18 decimals
assert liquidity_delta == adds.at[0, 'liquidity']

token0 = adds.at[0, 'token_0_amount']
token1 = adds.at[0, 'token_1_amount']
tick_lower = adds.at[0, 'price_tick_lower']
tick_upper = adds.at[0, 'price_tick_upper']
token0_decimals = adds.at[0, 'contract_decimals_token_0']
token1_decimals = adds.at[0, 'contract_decimals_token_1']

init_price = pool_init_price(token0, token1, tick_upper, tick_lower, liquidity_delta,
                             token0_decimals, token1_decimals)

token0_symb = liquidity.at[0, 'contract_ticker_symbol_token_0']
token1_symb = liquidity.at[0, 'contract_ticker_symbol_token_1']
print(f'Pool initial price ({token1_symb}/{token0_symb}): {init_price:,.12e}')
print(f'Pool initial price ({token0_symb}/{token1_symb}): {1 / init_price:,.12e}')

Pool initial price (WETH/USDC): 2.904162146572e+08
Pool initial price (USDC/WETH): 3.443333910196e-09


In [9]:
sqrt_price_x96 = 1.350174849792634181862360983626536e+33
etherscan_price = sqrt_price_x96 ** 2 / 2 ** 192
print(f"Calculated initial price: {init_price:.12e}")
print(f"Price per Etherscan:      {etherscan_price:.12e}")

Calculated initial price: 2.904162146572e+08
Price per Etherscan:      2.904162146577e+08


In [10]:
cols = ['tx_hash', 'txn_time', 'liquidity_event']
liqu_txn = liquidity.loc[:, cols].copy()
liqu_txn.reset_index(drop=False, inplace=True)
liqu_txn.rename(columns={'liquidity_event': 'event', 'index': 'orig_idx'}, inplace=True)

cols = ['tx_hash', 'swap_time']
swap_txn = swaps.loc[:, cols].copy()
swap_txn.reset_index(drop=False, inplace=True)
swap_txn.rename(columns={'swap_time': 'txn_time', 'index': 'orig_idx'}, inplace=True)
swap_txn['event'] = 'SWAP'

all_txn = pd.concat([liqu_txn, swap_txn], axis=0)
all_txn = all_txn.sort_values('txn_time').reset_index(drop=True)
all_txn.drop(all_txn.index[all_txn['txn_time'] > MAX_DATE], axis=0, inplace=True)

all_txn

,orig_idx,tx_hash,txn_time,event
0,31209,0x125e0b641d4a4b08806bf52c0c6757648c9963bcda86...,2021-05-05 21:42:11+00:00,ADD_LIQUIDITY
1,1247616,0x0804ff007263a885191f23c808a9346e62d502a1fc23...,2021-05-05 22:15:01+00:00,SWAP
2,31208,0x7258050fbb90fa41f73505c552a973113ffc87c67ddd...,2021-05-05 22:27:41+00:00,REMOVE_LIQUIDITY
3,31207,0x7258050fbb90fa41f73505c552a973113ffc87c67ddd...,2021-05-05 22:27:41+00:00,REMOVE_LIQUIDITY
4,31206,0xa564ea4677bda271bbb2eebec9774577b9b101c46eaf...,2021-05-05 22:44:11+00:00,ADD_LIQUIDITY
...,...,...,...,...
1268554,10187,0xc5a7f435798215884699b38c501029eb51d0523b571a...,2022-01-27 23:59:47+00:00,SWAP
1268555,10186,0xb2f99e85a17466bd82b9d035f7026dbd5b30616f7b7d...,2022-01-27 23:59:47+00:00,SWAP
1268556,10185,0x59a47e08919bc1b56aa94f75cd38ab6922bdb20f70ca...,2022-01-27 23:59:47+00:00,SWAP
1268557,10184,0x9a6c924a94242bb6c453dfe074d5f61834047a516702...,2022-01-27 23:59:47+00:00,SWAP


In [11]:
CHECKS_ON = True
# need to think about appropriate error tolerances
# TODO: maybe base these tolerances on the average transaction size?
TOKEN0_TOLS = {'atol': 1e-6, 'rtol': 1e-4}
TOKEN1_TOLS = {'atol': 1e-6, 'rtol': 1e-4}
LIQUIDITY_TOLS = {'atol': 1e-6, 'rtol': 1e-4}

In [12]:
all_txn = all_txn.iloc[:50].copy()

In [13]:
fee = liquidity.at[0, 'pool_fee'] / 1e+6
tick_spacing = liquidity.at[0, 'pool_tick_spacing']
pool = Uniswapv3Pool(fee, tick_spacing, init_price,
                     token0_decimals=token0_decimals,
                     token1_decimals=token1_decimals)
print(f'{pool}')

position_id = 'LP123'
results = []
for i, row in all_txn.iterrows():
    logging.info(f'Transaction {i}.')
    txn = row['event']
    idx = row['orig_idx']
    
    if 'LIQUIDITY' in txn:
        token0 = liquidity.at[idx, 'token_0_amount']
        token1 = liquidity.at[idx, 'token_1_amount']
        if txn == 'REMOVE_LIQUIDITY':
            token0 = -1 * token0
            token1 = -1 * token1
        
        tick_lower = liquidity.at[idx, 'price_tick_lower']
        tick_upper = liquidity.at[idx, 'price_tick_upper']
        liquidity_delta = liquidity.at[idx, 'liquidity']
        
        if pd.isnull(liquidity_delta):
            liquidity_delta = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price,
                token0_decimals,
                token1_decimals
            )
        elif CHECKS_ON:
            ld_calc = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price,
                token0_decimals,
                token1_decimals
            )
            assert np.isclose(liquidity_delta, ld_calc, **LIQUIDITY_TOLS), (
                f'Calculated liquidity_delta {ld_calc:,.12e} does '
                f'not match liquidity_delta per the data {liquidity_delta:,.12e}.'
            )
            
        position = pool.position_map[(position_id, tick_lower, tick_upper)]
        # If the liquidity_delta is very, very close to the position's total
        # liquidity, set liquidity_delta to the total liquidity to completely
        # close out the position
        if np.isclose(-position.liquidity, liquidity_delta):
            liquidity_delta = -position.liquidity
        # we also make sure that liquidity_delta cannot be less than the 
        # position's total liquidity
        if liquidity_delta < 0:
            liquidity_delta = max(liquidity_delta, -position.liquidity)
                
        token0_calc, token1_calc = pool.set_position(
            position_id, 
            tick_lower,
            tick_upper,
            liquidity_delta
        )
    elif txn == 'SWAP':
        token0 = swaps.at[idx, 'token_0_amount']
        token1 = swaps.at[idx, 'token_1_amount']
        
        token = 0 if token0 > 0 else 1
        tokens_in = token0 if token == 0 else token1
        
        token0_calc, token1_calc = pool.swap(token, tokens_in)
    else:
        raise ValueError(f'{txn} is not a valid transaction type.')
    
    if CHECKS_ON:
        assert np.isclose(token0, -token0_calc, **TOKEN0_TOLS), (
            f'Transaction {i:,}: token0 output {-token0_calc:,.12e} does not '
            f'match token0 in the data {token0:,.12e}.'
        )
        assert np.isclose(token1, -token1_calc, **TOKEN1_TOLS), (
            f'Transaction {i:,}: token1 output {-token1_calc:,.12e} does not '
            f'match token1 in the data {token1:,.12e}.'
        )
    
    results.append({
        'sqrt_price': pool.sqrt_price, 
        'liquidity': pool.liquidity
    })
    print(f'Completed transaction {i}.')

Pool(price=2.904162e-04, liquidity=0.000000e+00, fee=0.05%)
Completed transaction 0.
Completed transaction 1.
Completed transaction 2.
Completed transaction 3.
Completed transaction 4.
Completed transaction 5.
Completed transaction 6.
Completed transaction 7.
Completed transaction 8.
Completed transaction 9.
Completed transaction 10.
Completed transaction 11.
Completed transaction 12.
Completed transaction 13.
Completed transaction 14.
Completed transaction 15.
Completed transaction 16.
Completed transaction 17.
Completed transaction 18.
Completed transaction 19.
Completed transaction 20.
Completed transaction 21.
Completed transaction 22.
Completed transaction 23.
Completed transaction 24.
Completed transaction 25.
Completed transaction 26.
Completed transaction 27.
Completed transaction 28.
Completed transaction 29.
Completed transaction 30.
Completed transaction 31.
Completed transaction 32.
Completed transaction 33.
Completed transaction 34.
Completed transaction 35.
Completed tran